In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.FeatureCNN import FeatureCNN
from ai.DataLoader import DataLoader
from env import *

ETA = 1e-2
EPOCHS = 1000

print(torch.cuda.is_available())

True


In [2]:
train_dloader = DataLoader(TRASH_TRAIN_DATA_PATH, TRASH_CAT, noise=True)
valid_dloader = DataLoader(TRASH_VALID_DATA_PATH, TRASH_CAT, noise=True)

Number of data batch: 793
Number of batch: 100
Number of data batch: 65
Number of batch: 9


In [3]:
model = FeatureCNN()
model = model.cuda()

In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:
model.load(CNN_CKPT_PATH)

top_valid_acc = 0.0

for e in range(EPOCHS):
    
    train_loss = 0.0
    train_clf_acc = 0.0
    
    for x_batch_, y_batch_ in train_dloader.next_batch():
        x_batch_ = x_batch_.reshape(-1, IN_CHANNEL, HEIGHT, WIDTH)
        y_batch = np.repeat(y_batch_, 8, axis=0)
        
        x_batch = torch.FloatTensor(x_batch_).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        latent, logps = model(x_batch)
        loss = criterion(logps, y_batch)
        train_loss += loss.item()
        
        with torch.no_grad():
            ps = torch.exp(logps)
            val_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_clf_acc += torch.mean(equal.type(torch.FloatTensor))
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        
        valid_loss = 0.0
        valid_clf_acc = 0.0
        
        with torch.no_grad():
            model.eval()
            
            for x_batch_, y_batch_ in valid_dloader.next_batch():
                x_batch_ = x_batch_.reshape(-1, IN_CHANNEL, HEIGHT, WIDTH)
                y_batch = np.repeat(y_batch_, 8, axis=0)
                
                x_batch = torch.FloatTensor(x_batch_).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                latent, logps = model(x_batch)
                ps = torch.exp(logps)
                _, topk = ps.topk(1, dim=1)
                equal = topk == y_batch.view(*topk.size())
                valid_clf_acc += torch.mean(equal.type(torch.FloatTensor))
        
                loss = criterion(logps, y_batch)
                valid_loss += loss.item()

            train_loss /= len(train_dloader)
            train_clf_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_clf_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_clf_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_clf_acc:.8f}")
            
            if top_valid_acc < valid_clf_acc:
                top_valid_acc = valid_clf_acc
                model.save(CNN_CKPT_PATH)

            model.train()
        
# model.save(CNN_CKPT_PATH)

FeatureCNN was loaded.
Epochs: 10/1000
Train loss: 0.26553818
Train acc: 0.90933591
Valid loss: 0.70780984
Valid acc: 0.79340279
FeatureCNN was saved.
Epochs: 20/1000
Train loss: 0.21528077
Train acc: 0.92808592
Valid loss: 0.58851199
Valid acc: 0.80555558
FeatureCNN was saved.
Epochs: 30/1000
Train loss: 0.20298193
Train acc: 0.93671876
Valid loss: 0.51868378
Valid acc: 0.84809029
FeatureCNN was saved.
Epochs: 40/1000
Train loss: 0.14366819
Train acc: 0.95316404
Valid loss: 0.44810196
Valid acc: 0.89192706
FeatureCNN was saved.
Epochs: 50/1000
Train loss: 0.10032926
Train acc: 0.96753907
Valid loss: 0.49570900
Valid acc: 0.88411456
Epochs: 60/1000
Train loss: 0.14908279
Train acc: 0.95609373
Valid loss: 0.37304176
Valid acc: 0.89843750
FeatureCNN was saved.
Epochs: 70/1000
Train loss: 0.08379078
Train acc: 0.97230470
Valid loss: 0.50962783
Valid acc: 0.88281250
Epochs: 80/1000
Train loss: 0.08083974
Train acc: 0.97457033
Valid loss: 0.43672332
Valid acc: 0.92230904
FeatureCNN was save

KeyboardInterrupt: 